In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.11.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.11.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.11.1-alpha"
#r "nuget: Microsoft.Bing.Search.WebSearch, 1.0.0"
#r "nuget: Microsoft.ML, 3.0.1"
#r "nuget: Microsoft.ML.FastTree, 3.0.1"
#r "nuget: Microsoft.Extensions.ML, 3.0.1"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Bing.WebSearch;
using System.ComponentModel;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
var bingSearchAPIKey = config["BingSearch:APIKey"];

Installed Packages Microsoft.Bing.Search.WebSearch, 1.0.0 Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.Extensions.ML, 3.0.1 Microsoft.ML, 3.0.1 Microsoft.ML.FastTree, 3.0.1 Microsoft.SemanticKernel, 1.11.1 Microsoft.SemanticKernel.Plugins.Core, 1.11.1-alpha Microsoft.SemanticKernel.Plugins.Web, 1.11.1-alpha

In [3]:
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName,
        endpoint: azureOpenAIEndpoint,
        apiKey: azureOpenAIAPIKey)
    .Build();

// Import the BaseballHallOfFamePlugin.cs file
#!import ../Classes/BaseballHallOfFamePlugin.cs

// Add the native plugin with the three native functions to the semantic kernel
semanticKernel.ImportPluginFromType<BaseballHallOfFamePlugin>();

In [4]:
// Background Information on what is happening

// Directory of the historical baseball data
string filePathMLBBaseballBatters = Directory.GetParent(Directory.GetCurrentDirectory()) + "/Data/MLBBaseballBattersPositionPlayers.csv";
// Read the batters data from the CSV file
var batters = File.ReadAllLines(filePathMLBBaseballBatters)
                        .Skip(1)
                        .Select(v => MLBBaseballBatter.FromCsv(v))
                        .ToList();

// Machine Learing Model and Prediction Engine                       
string modelPath = Directory.GetParent(Directory.GetCurrentDirectory()) + "/Models/InductedToHoF-GeneralizedAdditiveModels.mlnet";
var mlContext = new MLContext(seed: 100);
ITransformer loadedModel;
DataViewSchema schema;

using (var stream = File.OpenRead(modelPath))
{
    loadedModel = mlContext.Model.Load(stream, out schema);
}
TransformerChain<ITransformer> transfomerChain = (TransformerChain<ITransformer>)loadedModel;

var predEngineInductedToHallOfFame = mlContext.Model.CreatePredictionEngine<MLBBaseballBatter, MLBHOFPrediction>(transfomerChain);

// Predict if Mike Trout will be inducted to the Hall of Fame
var mikeTroutStats = batters.Where(b => b.FullPlayerName == "Mike Trout").FirstOrDefault();
Console.WriteLine(mikeTroutStats.HR);
var prediction = predEngineInductedToHallOfFame.Predict(mikeTroutStats);
Console.WriteLine(prediction.Probability);

350
0.69909585


In [5]:
// 1) Use WebSearch to find information on Mike Trout

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Provide sources of information and explain your reasoning.
Ensure to cite all the sources using [number] notation of each URL after the reference in order.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on the available data and sources, I would recommend that Mike Trout is a strong candidate for induction into the Hall of Fame.

**Reasoning:**
1. **Statistical Achievement**: Mike Trout's statistics are highly impressive, showcasing his ability as a player. Over 12 years, he has achieved a .303 batting average, .587 slugging percentage, 350 home runs, and 896 RBIs [1]. These figures are comparable to many Hall of Famers.

2. **Awards and Recognitions**: Trout has won three MVP awards and has been an All-Star nine times. He also has a total of 21 player awards [1]. His consistent recognition at a national level underscores his exceptional talent and contribution to the sport.

3. **Comparative Analysis**: Trout has already surpassed many historical figures in baseball in terms of player statistics and impact, even at a relatively young age [7]. His performance metrics are on par or better than many who have been inducted into the Hall of Fame.

4. **Expert Opinions and Public Per

In [6]:
// 2) Use Baseball statistics to determine if Mike Trout should be inducted to the Hall of Fame

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Provide as many baseball statistics as possible.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on the available statistics for Mike Trout, here is a detailed breakdown that could help in determining his potential induction into the Hall of Fame:

- **Full Player Name:** Mike Trout
- **Years Played:** 12
- **At Bats (AB):** 5094
- **Runs (R):** 1052
- **Hits (H):** 1543
- **Doubles:** 296
- **Triples:** 51
- **Home Runs (HR):** 350
- **Runs Batted In (RBI):** 896
- **Stolen Bases (SB):** 204
- **Batting Average:** .303
- **Slugging Percentage:** .587
- **All-Star Appearances:** 9
- **Total Bases (TB):** 2991
- **Total Player Awards:** 21 (This could include MVPs, Silver Sluggers, etc.)
- **Last Year Played:** 2022

### Hall of Fame Consideration:
Mike Trout's statistics are impressive, especially in categories such as home runs, runs scored, batting average, and slugging percentage. His 9 All-Star appearances and 21 awards indicate a high level of recognition in the sport over a significant period. These accomplishments are strong indicators for Hall of Fame consideration.


In [7]:
// 3) Use Machine Learning Prediction to determine if Mike Trout should be inducted to the Hall of Fame

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the Hall of Fame? 
Use a machine learning model to get the probability.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on the machine learning model, Mike Trout has a 69.91% probability of being inducted into the Hall of Fame. This is a relatively high probability, suggesting that he has a strong case for induction based on his current statistics and accomplishments in the sport.

In [9]:
// 4) All of the tools

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 6000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Use all of the tools available at your disposal to arrive at a comprehensive decision.
You must cite all the sources using [number] notation of each URL after the reference in order.
The sources should appear in order on the bottom of the decision.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on the comprehensive analysis of Mike Trout's career statistics, achievements, and the expert opinions gathered from various sources, it is recommended that Mike Trout should be inducted into the Hall of Fame.

Mike Trout's career highlights include:
- 12 years of play with significant achievements [1].
- A batting average of .303, 350 home runs, and 896 RBIs [2].
- Nine All-Star appearances and numerous player awards [2].
- A high probability (69.91%) of induction to the Hall of Fame based on machine learning analysis of his statistics [3].

Furthermore, Trout is fully qualified for the Hall of Fame as per the eligibility rules [1], and his impact on the game is widely recognized by experts and analysts. Despite some concerns about his injury history [4], his overall performance and accolades make a strong case for his induction.

In conclusion, considering Mike Trout's outstanding contributions to baseball, his statistical record, and expert opinions, it is highly recommended t